In [29]:
import mediapipe as mp
import cv2 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  
import pickle 
import time

In [30]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

In [45]:


with open('modelos/temporalidad_1.pkl', 'rb') as f:
    model = pickle.load(f)
print("modelo cargado")

time_limit = 20
start_time = time.time()

def max(probs):
    class_counts = {}
    # Contar la frecuencia de cada clase
    for result in probs:
        class_name = result.get("Class:")
        if class_name in class_counts:
            class_counts[class_name] += 1
        else:
            class_counts[class_name] = 1
    
    # Encontrar la clase con la mayor frecuencia
    max_count = 0
    max_class = None
    for class_name, count in class_counts.items():
        if count > max_count:
            max_count = count
            max_class = class_name
    
    # Buscar el resultado con la probabilidad más alta dentro de la clase más repetida
    max_prob = float('-inf')
    for result in probs:
        if result.get("Class:") == max_class:
            prob = result.get("Prob:", float('-inf'))
            if prob > max_prob:
                max_prob = prob
    
    return max_class, max_prob


cap = cv2.VideoCapture(0)
probs=[]
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.6) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        #print("leyendo video")
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        #print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            
            # Make Detections
            X = pd.DataFrame([row])
            #print(type(X))
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            #print(body_language_prob)
            max_prob_index = np.argmax(body_language_prob)
            probs.append({"Class:": body_language_class, "Prob:": body_language_prob[max_prob_index]})


            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, ''
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, ''
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Rosa Webcam', image)

        if time.time() - start_time > time_limit:
            break

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
print(probs)
print(f"Total probs: {len(probs)}")
#max_class, max_prob = max(probs)
#print(f"MaxClass: '{max_class}',MaxProb:  '{max_prob}'")
print(f"Clase mas repetida: '{max(probs)}'")

cap.release()
cv2.destroyAllWindows()

modelo cargado


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have va

[{'Class:': 'ayer', 'Prob:': 0.76}, {'Class:': 'ayer', 'Prob:': 0.7}, {'Class:': 'ayer', 'Prob:': 0.7}, {'Class:': 'ayer', 'Prob:': 0.74}, {'Class:': 'ayer', 'Prob:': 0.72}, {'Class:': 'ayer', 'Prob:': 0.63}, {'Class:': 'ayer', 'Prob:': 0.5}, {'Class:': 'ayer', 'Prob:': 0.58}, {'Class:': 'ayer', 'Prob:': 0.49}, {'Class:': 'ayer', 'Prob:': 0.46}, {'Class:': 'ayer', 'Prob:': 0.49}, {'Class:': 'ayer', 'Prob:': 0.5}, {'Class:': 'ayer', 'Prob:': 0.48}, {'Class:': 'ayer', 'Prob:': 0.48}, {'Class:': 'ayer', 'Prob:': 0.48}, {'Class:': 'dia', 'Prob:': 0.44}, {'Class:': 'ayer', 'Prob:': 0.49}, {'Class:': 'ayer', 'Prob:': 0.5}, {'Class:': 'ayer', 'Prob:': 0.5}, {'Class:': 'ayer', 'Prob:': 0.53}, {'Class:': 'ayer', 'Prob:': 0.55}, {'Class:': 'ayer', 'Prob:': 0.55}, {'Class:': 'ayer', 'Prob:': 0.54}, {'Class:': 'ayer', 'Prob:': 0.52}, {'Class:': 'ayer', 'Prob:': 0.54}, {'Class:': 'ayer', 'Prob:': 0.41}, {'Class:': 'dia', 'Prob:': 0.46}, {'Class:': 'ayer', 'Prob:': 0.47}, {'Class:': 'dia', 'Prob:': 

: 